데이터 형태 예제
- CPU:  Intel(R) Xeon(R) Gold 6326 CPU @ 2.90GHz 64 core
- MEM:  DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB, DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB,DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB
- DISK:  SMC3108 SSD 893.8G, SMC3100 SSD 500G
- NETWORK: Intel Corporation Ethernet Controller X710 for 10GBASE-T (rev 02)

In [5]:
import subprocess
import re

from cpuinfo import get_cpu_info

In [6]:
# CPU
cpu_info = get_cpu_info()
cpu = f"{cpu_info['brand_raw']} {cpu_info['count']} core"
cpu

'Intel(R) Xeon(R) Gold 6326 CPU @ 2.90GHz 64 core'

In [11]:
# Memory
output = subprocess.check_output(["sudo", "lshw", "-c", "memory"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("*-") if i[:4] == "bank"]
memory = []

for line in lines:
    pattern_description = re.compile(r"description: (.+)\n")
    pattern_size = re.compile(r"size: (.+)\n")
    
    description = pattern_description.search(line).group(1)
    if "empty" in description:
        continue
    size = pattern_size.search(line).group(1)

    memory.append(f"{description} {size}")

memory

['DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
 'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB']

In [10]:
# Disk
output = subprocess.check_output(["sudo", "lshw", "-c", "disk"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("*-") if i.strip()]
disks = []

for line in lines:
    pattern_description = re.compile(r"description: (.+)\n")
    pattern_product = re.compile(r"product: (.+)\n")
    pattern_capacity = re.compile(r"capacity: (.+)\n")
    pattern_size = re.compile(r"size: (.+)\n")

    description = pattern_description.search(line).group(1) if pattern_description.search(line) else None
    product = pattern_product.search(line).group(1) if pattern_product.search(line) else None
    capacity = pattern_capacity.search(line).group(1) if pattern_capacity.search(line) else None
    size = pattern_size.search(line).group(1) if pattern_size.search(line) else None

    disk = f"[{description}]"
    if product:
        disk += f" {product}"
    if capacity:
        disk += f" {capacity}"
    elif size:
        disk += f" {size}"
    disks.append(disk)

disk

'[SCSI Disk] SMC3108 893GiB (959GB)'

In [11]:
# Network
output = subprocess.check_output(["sudo", "lshw", "-c", "network"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("*-") if i.strip()]
network = []

for line in lines:
    pattern_product = re.compile(r"product: (.+)\n")
    
    searched = pattern_product.search(line)
    if not searched:
        continue
    product = searched.group(1)
    
    if not product in network:
        network.append(f"{product}")

network

['MT28908 Family [ConnectX-6]', 'Ethernet Controller X710 for 10GBASE-T']

In [36]:
# HCA
output = subprocess.check_output(["sudo", "ibstat"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("CA ") if i.strip()]
hcas = []

for idx, line in enumerate(lines[::2]):
    hca = line.strip("'\n\t")
    info = lines[idx * 2 + 1]
    if "State: Active" in info and "Physical state: LinkUp" in info and "Link layer: InfiniBand" in info:
        hcas.append(hca)

hcas

['mlx5_2', 'mlx5_3']

In [1]:
from kubernetes import client, config
from app.get_info import *
from datetime import datetime

In [2]:
# Load kubernetes config
cfg = config.load_kube_config()

# Create kubernetes client
k8s_client = client.ApiClient(cfg)

# Create kubernetes v1
v1 = client.CoreV1Api(k8s_client)

node_name = "k8s-node-2"

In [3]:
hardware_info = {
    "updated_datetime": datetime.now().isoformat(),
    "cpu": cpu_info(),
    "mem": memory_info(sudo=True),
    "disk": disk_info(sudo=True),
    "network": network_info(sudo=True),
    "hca": hca_info(),
}

hardware_info

{'updated_datetime': '2023-09-01T04:52:45.170748',
 'cpu': 'Intel(R) Xeon(R) Gold 6326 CPU @ 2.90GHz 64 core',
 'mem': ['DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB'],
 'disk': ['[SCSI Disk] SMC3108 893GiB (959GB)'],
 'network': ['MT28908 Family [ConnectX-6]',
  'Ethernet Controller X710 for 10GBASE-T'],
 'hca': ['mlx5_2', 'mlx5_3']}

In [4]:
# Get node annotations
node = v1.read_node(name=node_name)

In [5]:
node

{'api_version': 'v1',
 'kind': 'Node',
 'metadata': {'annotations': {'flannel.alpha.coreos.com/backend-data': '{"VNI":1,"VtepMAC":"f6:cd:37:dc:81:e7"}',
                              'flannel.alpha.coreos.com/backend-type': 'vxlan',
                              'flannel.alpha.coreos.com/kube-subnet-manager': 'true',
                              'flannel.alpha.coreos.com/public-ip': '101.202.0.9',
                              'gpu_uuid': '{}',
                              'hardware-info': '{"updated_datetime": '
                                               '"2023-09-01T04:51:32.889490", '
                                               '"cpu": "Intel(R) Xeon(R) Gold '
                                               '6326 CPU @ 2.90GHz 64 core", '
                                               '"memory": ["DIMM DDR4 '
                                               'Synchronous Registered '
                                               '(Buffered) 3200 MHz (0.3 ns) '
                

In [6]:
# Cast to string for input annotations, Replace single quote for reading with json.loads()
hardware_info = str(hardware_info).replace("'", '"')
node.metadata.annotations["hardware-info"] = hardware_info

In [7]:
# Patch node annotations
v1.patch_node(name=node_name, body=node)

{'api_version': 'v1',
 'kind': 'Node',
 'metadata': {'annotations': {'flannel.alpha.coreos.com/backend-data': '{"VNI":1,"VtepMAC":"f6:cd:37:dc:81:e7"}',
                              'flannel.alpha.coreos.com/backend-type': 'vxlan',
                              'flannel.alpha.coreos.com/kube-subnet-manager': 'true',
                              'flannel.alpha.coreos.com/public-ip': '101.202.0.9',
                              'gpu_uuid': '{}',
                              'hardware-info': '{"updated_datetime": '
                                               '"2023-09-01T04:52:45.170748", '
                                               '"cpu": "Intel(R) Xeon(R) Gold '
                                               '6326 CPU @ 2.90GHz 64 core", '
                                               '"mem": ["DIMM DDR4 Synchronous '
                                               'Registered (Buffered) 3200 MHz '
                                               '(0.3 ns) 32GiB", "DIMM DDR4 '


In [8]:
import json

node = v1.read_node(name=node_name)

hw_info = json.loads(node.metadata.annotations["hardware-info"])

In [9]:
hw_info

{'updated_datetime': '2023-09-01T04:52:45.170748',
 'cpu': 'Intel(R) Xeon(R) Gold 6326 CPU @ 2.90GHz 64 core',
 'mem': ['DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB',
  'DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB'],
 'disk': ['[SCSI Disk] SMC3108 893GiB (959GB)'],
 'network': ['MT28908 Family [ConnectX-6]',
  'Ethernet Controller X710 for 10GBASE-T'],
 'hca': ['mlx5_2', 'mlx5_3']}